# DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
y2015 = pd.read_csv(
    'C:/Users/Carter Carlson/Documents/Thinkful/Large Databases/Loan Data.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [4]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_May-2018,settlement_date_Nov-2015,settlement_date_Nov-2016,settlement_date_Nov-2017,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017
0,NaN,NaN,20000.0,20000.0,20000.0,9.80,643.47,75000.0,23.45,0.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,15000.0,15000.0,15000.0,13.44,344.69,77213.0,18.17,0.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,7200.0,7200.0,7200.0,11.48,237.36,72500.0,8.47,0.0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,15000.0,15000.0,15000.0,8.49,473.45,110000.0,13.24,0.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,16000.0,16000.0,16000.0,8.49,328.19,62000.0,28.92,0.0,...,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,6000.0,6000.0,6000.0,7.49,186.61,100000.0,13.28,1.0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,20000.0,20000.0,20000.0,15.77,700.88,70000.0,22.21,0.0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,15000.0,15000.0,15000.0,5.32,451.73,80000.0,20.18,0.0,...,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,14000.0,14000.0,14000.0,9.80,450.43,100000.0,15.97,0.0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,NaN,3600.0,3600.0,3600.0,11.48,118.68,85000.0,17.44,0.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.98976539, 0.9896704 , 0.98895749, 0.98860128, 0.99042983,
       0.98898098, 0.98442138, 0.9876983 , 0.97330674, 0.9841594 ])